# Fuzzy logic

This workbook explores the concepts of **fuzzy logic** and **fuzzy rules** in Python. 

Throughout this notebook, you will use scikit-fuzzy, a python library in the SciPy stack, which integrates well with numpy and scipy.You will find some guided examples to aid your understanding, and some exercises for you to implement on your own.

#### Content:
* [skfuzzy](#skfuzzy)
    * [Getting started](#skfuzzy-start)
    * [Exercise ](#skfuzzy-ex1)


## skfuzzy <a class="anchor" id="skfuzzy"></a>

**[skfuzzy](https://scikit-fuzzy.github.io/scikit-fuzzy/overview.html)** is a Python library to create fuzzy logic systems, offering tools to implement fuzzy sets, fuzzy rules, and fuzzy inference, in particular suitable to create fuzzy control systems efficiently.

Let's start!

### Getting started <a class="anchor" id="skfuzzy-start"></a>

In [ ]:
# uncomment the cells below to install skfuzzy
# ! pip install --upgrade pip
# ! pip install scikit-fuzzy

In [ ]:
import numpy as np
import skfuzzy as fuzz
import matplotlib.pyplot as plt 

In what follow we are going to implement a simply heating control system, that use the current value for temperature and humidity to decide on the heating power.

We first need to define the **linguistic variables**: temperature, humidity (inputs) and heater power (output), and we associate to each of them a **physical domain**. In skfuzzy, inputs are _antecedents_, and outputs are _consequents_.

##### Inputs

In [ ]:
# Define the input variables
# temperature is in celsius unit
# Humidity is a percentage value

from skfuzzy import control as ctrl 

temperature = fuzz.control.Antecedent(np.arange(0, 41, 0.1), 'temperature')
humidity = fuzz.control.Antecedent(np.arange(0, 101, 1), 'humidity')

We need to define the **linguistic values** for each input, and the corresponding **membership functions**. 

Linguistic values:
* temeperature: (cold, medium, warm)
* humidity: (dry, normal, wet) 

Membership functions:

For simplicity we will consider **triangular** and **trapezoid** membership functions. In other words, each interval of the linguistic values will be represented by a straight line, creating a trangle or a trapezoid for the whole variable physical domain. In skfuzzy this is done by using `trimf` for triangular functions, and `trapmf` for trapezoid functions.

In [ ]:
# Define the membership functions for temperature
# the triplet [a,b,c] in each function
# represents the 3 edges of the triangle 

temperature['cold'] = fuzz.trimf(temperature.universe, [0, 0, 20 ])
temperature['medium'] = fuzz.trimf(temperature.universe, [15, 25, 35])
temperature['warm'] = fuzz.trimf(temperature.universe, [25, 40, 40])


# Define the membership functions for temperature
humidity['dry'] = fuzz.trimf(humidity.universe,[0,0,40])
humidity["normal"] = fuzz.trapmf(humidity.universe,[30,45,75,90])
humidity["wet"] = fuzz.trimf(humidity.universe,[70,100,100])

Let's visualise the memberships functions defined.

In [ ]:
temperature.view()

In [ ]:
humidity.view()

##### Outputs

We define now the output of our fuzzy system, heater power, with the following characteristics. 

- Physical domain:we will consider the power as percentage of the overall heating power

- Linguistic values: Low, Moderate, High

- Membership functions: Triangular membership function

- Defuzzification: for the output variable we need to specify the method to translate the fuzzy inferenced value into the actual crisp value used by the system to regulate the temperature. We will use the **centroid method**.

In [ ]:
# Define output variable
heater_power = ctrl.Consequent(np.arange(0,100,0.1) , "power" , defuzzify_method='centroid')

# Define the membership functions for heater power
heater_power["low"] = fuzz.trimf(heater_power.universe,[0,0,40])
heater_power["moderate"] = fuzz.trimf(heater_power.universe,[20,50,80])
heater_power["high"] = fuzz.trimf(heater_power.universe,[60,100,100])


heater_power.view()

##### Rules

We need to define the rules of our fuzzy heating system, combining all the possible status for temperature and humidity. For example

<center> 
    If temperature is <b>cold</b> and humidity is <b>wet</b>, then heater is at <b>high</b> power.
</center>
    
Since each of our input variable has 3 status, there are 3x3=9 possible combinations, hence 9 rules to define. These rules can simply be visualise as in the table below:

<figure>
<img src="heater_rules.png" alt="Heater rules table" style="width: 500px;"/> 
</figure>

In [ ]:
# Define fuzzy rules

# heater power output = high
rule1 = ctrl.Rule(temperature["cold"] & humidity["wet"] , heater_power["high"])
rule2 = ctrl.Rule(temperature["cold"] & humidity["normal"] , heater_power["high"])
rule3 = ctrl.Rule(temperature["medium"] & humidity["wet"] , heater_power["high"])

# heater power output = moderate
rule4 = ctrl.Rule(temperature["cold"] & humidity["dry"] , heater_power["moderate"])
rule5 = ctrl.Rule(temperature["medium"] & humidity["normal"] , heater_power["moderate"])
rule6 = ctrl.Rule(temperature["warm"] & humidity["wet"] , heater_power["moderate"])

# heater power output = low
rule7 = ctrl.Rule(temperature["medium"] & humidity["dry"] , heater_power["low"])
rule8 = ctrl.Rule(temperature["warm"] & humidity["dry"] , heater_power["low"])
rule9 = ctrl.Rule(temperature["warm"] & humidity["normal"] , heater_power["low"])

In [ ]:
# Create a fuzzy system with the rules just defined
# this is our fuzzy knowledge base
heater_ctrl = ctrl.ControlSystem([rule1,rule2,rule3,rule4,rule5,rule6,rule7,rule8,rule9])

# Create the simulation
# this is our 'inference' engine
heater = ctrl.ControlSystemSimulation(heater_ctrl)

##### In action!
It's time to use our system. Let's input some value for temperature and humidity.

In [ ]:
# Input values
# T=18*C, H=60%
heater.input['temperature'] = 18
heater.input['humidity'] = 60

# Get the output
heater.compute()
output = heater.output['power']

print('Heating power: {:.2f}%'.format(output))

Let's got through the **fuzzification** and **defuzzification** processes happening above.


##### Fuzzyfication

When inputing the values, the system computes the corresponding membership function values for T=18C and  H=60% (fuzzyfication). We can compute the **degree** of membership for each variable status (=linguistic values) using `interp_membership`. This functions takes 3 arguments: the physical domain, the membership function for the specific status, and the input value.

In [ ]:
# Membership degree values for T=18*C
t = 18
print('Temperature input: {}C'.format(t))
for status in ['cold', 'medium', 'warm']:
    # compute membership degree
    degree = fuzz.interp_membership(temperature.universe, temperature[status].mf, t)
    print('Degree for {}: {}'.format(status, degree))

# Membership degree values for H=60%
h=60
print('\nHumidity input: {}%'.format(h))
for status in ['dry', 'normal', 'wet']:
    # compute membership degree
    degree = fuzz.interp_membership(humidity.universe, humidity[status].mf, h)
    print('Degree for {}: {}'.format(status, degree))

We can visualise the values above adding our simulation/inference engine to the `.view()` of temperature and humidity.

In [ ]:
# Temperature fuzzification
temperature.view(sim=heater)

In [ ]:
# Humidity fuzzification
print(humidity.view(sim=heater))

In the above plots, the black line is drawn at the input value.

##### Rules

Given the degree outputs obtained above, only the following rules are considered by the sistem:

- rule 2: if temperature is cold **AND** humidity is normal, then heater power is **high**
- rule 5: if temperature is medium **AND** humidity is normal, then heater power is **moderate**

As we have seen during Week 6 lecture, for two membership functions f1, f2, the intersection degree (**AND**) is given by `min(f1,f2)`. 

Given:

- Degree for temperature cold: 0.1
- Degree for temperature medium: 0.3
- Degree for temperature normal: 1.0

We obtain the following degree intersection:
- rule 2: min(0.1, 1) = 0.1
- rule 5: min(0.3, 1) = 0.3

Hence
- rule 2: activation heater power **0.1 high**
- rule 5: activation heater power **0.3 moderate**

##### Defuzzyfication
 
We need to translate the active fuzzy rules outputs into a crisp value output. In defining `heater_power` we have specified the defuzzify method to be **centroid**, hence we need to compute the 'central' output among those activated. 


Each rule adds constraints to the original membership function, restricting the possible values, reducing the _fuzzy set_ for moderate and high. 

The plot below outputs which parts of the `heater_power` membership functions is valid after reasoning using our inference engine and inputs.

In [ ]:
heater_power.view(sim=heater)

The plot above shows some coloured areas:
- orange: this is the area corresponding to rule5, this is the new fuzzy set containing the valid values for 'moderate'
- green: this is the area corresponding to rule2, this is the new fuzzy set containing the valid values for 'high'

Which value to choose then? Well, any power value within the highlighted areas is a valid answer. However, we need a crisp value to make sure our heating system works. To find such unique value, we can use the **centroid method**. This method computes the **centre** of the overall highlighted area (= the fuzzy set of all possible power values, given our two rules).


We will not go into the details of the formula, since this involves concepts not in scope for this module, however a detailed explanation with example can be found [in this website](https://codecrucks.com/center-of-gravity-method-for-defuzzification/).

Applying the centroid formula, our inference engine outputs a power value of 54.81%; this is highlighted in the plot by the black vertical line.

Finally, given our inputs of T=18C and H=60%, we now know that our heating system will be activated at 54.81% of its power!


### Exercise -  smart traffic light <a class="anchor" id="skfuzzy-ex"></a>

You have been tasked with creating a control system for the traffic light at a specif junction stop.

The duration of the green light will depend on 3 factors:
- waiting_traffic: current number of vehicles waiting at the stop
- oncoming_traffic: number of vehicles arriving to the stop (from other traffic lanes)
- pedestrian_density: number of pedestrian  need to cross the junction while traffic red light is on

**Task1**

Create the inputs and otputs of your fuzzy logic system, with the following linguistic values:

Input:
- waiting_traffic:  
    - values: ['free_flow', 'light', 'moderate', 'heavy', 'standstill']
    - domain: (0,100)
- oncoming_traffic: 
    - values: ['minimal', 'low', 'average', 'high']
    - domain: (0,100)

- pedestrian_density:
    - values: ['low', 'moderate', 'high']
    - domain: (0,10)

Output:
- light_duration:  
    - values: ['short', 'medium', 'long']
    - domain (unit=seconds): (0,180)

For each variable/linguistic value you will need to define an appropriate membership function. You can do this manually, or make skfuzzy find the most appropriate functions using:
`<variable name>.automf(<number_of_linguistic_values>, names=<linguistic_values>)`

In [ ]:
# write here your code

**Task2**

Create the rules of your traffic light control.

Given the number of linguistic values defined above for the input variables, there are 5x4x3=60 possibilities, hence 60 rules need to be defined. You can define these rules one by one, or you can write a function that creates all the 60 possibilities, and randomly associated one state of light_duration to each rule.

In [ ]:
# write here your code

**Task3**

It's time to control the traffic lights! Create the KB and inference engine.

How long will the red light last in the following situation:

- Waiting traffic: 50 cars
- Oncoming traffic: 10 cars
- Pedestrian: 0

- Waiting traffic: 0 cars
- Oncoming traffic: 17 cars
- Pedestrian: 3

- Waiting traffic: 90 cars
- Oncoming traffic: 0 cars
- Pedestrian: 2

In [ ]:
# write here your code